# Fetch Assessment

In [1]:
from utils import *
from dataset import *

In [2]:
dataset = TaskDataset(data=None)

Using cache found in C:\Users\yashv/.cache\torch\hub\huggingface_pytorch-transformers_main


# Task 1
Sentence Embedding Model, architecture:
 - Embedding Layer 
 - Positional Encoding
 - Transformer Encoder Layer
 - Mean Pooling
 - Linear Layer (generates embedding)

In [12]:
# initialize models
n_layers = 1 # The number of transformer encoder layers
d_model = 512 # The input embedding dimension
embed_size = 300 # The output embedding size
nhead = 8 # The number of attention heads in encoder layer
task_type = "se" # The type of task (sentence embedding or sentiment classification)
vocab_size = dataset.vocab_size # The size of the vocabulary (using hugging face bert tokenizer)
base_model, task_model = init_model(vocab_size, n_layers, d_model, nhead, embed_size, task_type)

In [13]:
# Generate sample input
input_data = torch.randint(0, dataset.vocab_size, (2, 100))

# Forward pass
base_output = base_model(input_data)
task_output = task_model(base_output)

print("Base model output:", base_output.shape)
print("Task model output:", task_output.shape)

# Generate input based on text
text1 = "How, are you?"
text2 = "I am good"

input_data1 = dataset.encode(text1)
input_data2 = dataset.encode(text2)

# Forward pass
base_output1 = base_model(input_data1)
base_output2 = base_model(input_data2)
task_output1 = task_model(base_output1)
task_output2 = task_model(base_output2)

print("Base model output 1:", base_output1.shape)
print("Base model output 2:", base_output2.shape)
print("Task model output 1:", task_output1.shape)   
print("Task model output 2:", task_output2.shape)
print("Task model output 2:", task_output2)

Base model output: torch.Size([2, 100, 512])
Task model output: torch.Size([2, 300])
Base model output 1: torch.Size([1, 7, 512])
Base model output 2: torch.Size([1, 5, 512])
Task model output 1: torch.Size([1, 300])
Task model output 2: torch.Size([1, 300])
Task model output 2: tensor([[-0.2726, -0.2454, -0.7493, -0.4067,  0.4283,  0.2488, -0.0580, -0.1344,
          0.1117,  0.3108, -0.2256, -0.1204, -0.4034,  0.9493, -0.1398,  0.0463,
         -0.2360,  0.4546,  0.7274,  0.5364, -0.8007, -0.3922,  0.1981, -0.1664,
         -0.4728, -0.1081, -0.0227,  0.3971,  0.6595, -0.3381,  0.0288, -0.8420,
          0.6615, -0.5636, -0.3395,  0.1633,  0.5974, -0.6506,  0.4265, -0.0961,
         -0.2653,  0.2150, -0.3743, -0.5631, -0.1082, -0.3951, -0.8663,  0.0406,
         -0.6262,  0.2172, -0.1110, -0.3706, -0.3270,  0.0354,  0.3387,  0.1499,
          0.3395,  0.4351, -0.2827,  0.1397,  0.2028, -0.4214,  0.4366, -0.3335,
         -0.0480,  0.1418,  0.0043,  0.2354, -0.1562,  0.3049,  0.1841, 

# Task 2
Sentiment Classification Model, architecture:
 - Embedding Layer
 - Positional Encoding
 - Transformer Encoder Layer
 - Mean Pooling
 - Linear Layer (3 classes, Positive, Negative, Neutral) ( can be trained using cross entropy loss)

In [5]:
task_type = "sc"
base_model, task_model = init_model(vocab_size, n_layers, d_model, nhead, embed_size, task_type)

In [10]:
# Generate sample input
input_data = torch.randint(0, dataset.vocab_size, (2, 100))

# Forward pass
base_output = base_model(input_data)
task_output = task_model(base_output)
task_output = nn.Softmax(dim=1)(task_output).detach().numpy()

print("Base model output:", base_output.shape)
print("Task model output:", task_output.shape)
print("Task model output:", task_output)

# Generate input based on text
text1 = "How, are you?"
text2 = "I am good"

input_data1 = dataset.encode(text1)
input_data2 = dataset.encode(text2)

# Forward pass
base_output1 = base_model(input_data1)
base_output2 = base_model(input_data2)
task_output1 = task_model(base_output1)
task_output2 = task_model(base_output2)

task_output1 = nn.Softmax(dim=1)(task_output1).detach().numpy()
task_output2 = nn.Softmax(dim=1)(task_output2).detach().numpy()

print("Base model output 1:", base_output1.shape)
print("Base model output 2:", base_output2.shape)
print("Task model output 1:", task_output1.shape)   
print("Task model output 2:", task_output2.shape)

Base model output: torch.Size([2, 100, 512])
Task model output: (2, 3)
Task model output: [[0.24638513 0.22836564 0.52524924]
 [0.24191225 0.2560569  0.5020308 ]]
Base model output 1: torch.Size([1, 7, 512])
Base model output 2: torch.Size([1, 5, 512])
Task model output 1: (1, 3)
Task model output 2: (1, 3)
